# Аналитика в Яндекс.Афише

Цель исследования: оптимизировать маркетинговые затраты.<br>
Оптимизацию возможно провести путем выбора оптимальных рекламного источника, размера затрат и платформы. <br>
Для анализа будут использоваться данные за 2017-2018 гг. по визитам на сайт, заказам и маркетинговым затратам.<br>

План: <br>
1 Загрузка и подготовка данных<br>
2 Посторение отчетов и расчет метрик<br>
3 Общий вывод<br>

## 1 Загрузка и подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
visits = pd.read_csv('https://code.s3.yandex.net/datasets/visits_log.csv')
orders = pd.read_csv('https://code.s3.yandex.net/datasets/orders_log.csv')
costs = pd.read_csv('https://code.s3.yandex.net/datasets/costs.csv')

In [ ]:
visits

In [ ]:
visits.info()

In [ ]:
orders

In [ ]:
orders.info()

In [ ]:
costs

In [ ]:
costs.info()

In [ ]:
visits.columns=['device', 'end_ts', 'source_id', 'start_ts', 'uid']
orders.columns=['buy_ts', 'revenue', 'uid']

In [ ]:
visits['end_ts'] = pd.to_datetime(visits['end_ts'], format="%Y-%m-%d %H:%M")
visits['start_ts'] = pd.to_datetime(visits['start_ts'], format="%Y-%m-%d %H:%M")
orders['buy_ts'] = pd.to_datetime(orders['buy_ts'], format="%Y-%m-%d %H:%M")
costs['dt'] = pd.to_datetime(costs['dt'], format="%Y-%m-%d %H:%M")

In [ ]:
visits['session_year']  = visits['start_ts'].dt.year
visits['session_month'] = visits['start_ts'].dt.month
visits['session_week']  = visits['start_ts'].dt.week
visits['session_date'] = visits['start_ts'].dt.date

orders['buy_year']  = orders['buy_ts'].dt.year
orders['buy_month'] = orders['buy_ts'].dt.month
orders['buy_week']  = orders['buy_ts'].dt.week

costs['cost_year']  = costs['dt'].dt.year
costs['cost_month'] = costs['dt'].dt.month
costs['cost_week']  = costs['dt'].dt.week

<div style="border:solid grey 2px; padding: 30px">
В таблицах переименовали поля. Перевели дату в dt. Выделили из нее недели, месяцы, года.

Соединяем нужные таблицы

In [ ]:
#sources_uids
uids = visits.sort_values('start_ts').groupby('uid')[['source_id', 'device']].agg('first').reset_index()
sources_count = uids.groupby('source_id')['uid'].count().reset_index()

In [ ]:
orders = orders.merge(uids, on='uid')

## 2 Построение отчетов и метрик

## Смотрим, что сразу можно увидеть в данных

In [ ]:
print(f'Количество визитов за период {visits["uid"].count()}')
print(f'Количество уникальных пользователей за период {visits["uid"].nunique()}')
print()
print('Распределение визитов по устройствам')
display(visits['device'].value_counts(normalize=True))
print()
print('Колиество визитов по источникам рекламы')
display(visits['source_id'].value_counts())

In [ ]:
print('Диапазон дат визитов')
visits['start_ts'].describe()

<div style="border:solid grey 2px; padding: 30px">В период с июня 2017 г. по май 2018 г. 228 тыс. уникальных пользователей сгенерировали 359 тыс. сессий. Большая часть визитов (73%) c десктопа. Из источников 7, 6 почти нет переходов.

In [ ]:
print(f'Количество заказов за период {orders["uid"].count()}')
print(f'Количество пользователей, совершивших заказ за период {orders["uid"].nunique()}')
print()
print('Диапазон дат заказов')
display(orders['buy_ts'].describe())
print()
print(f'Общая сумма заказов {orders["revenue"].sum()}')
print()
print('Распеределение выручки по устройствам')
display(orders.groupby('device')['revenue'].sum())
print()
print('Распеределение выручки по источникам рекламы')
display(orders.groupby('source_id')['revenue'].sum())

<div style="border:solid grey 2px; padding: 30px">
За этот же период 37 тыс. пользователей было сделано 50 тыс. заказов, выручка Яндекс.Афиши по которым составила 252 млн руб. <br>
Большая часть покупок была сделана пользователям с десктопа. Выручка примерно равномерно распределена по источникам рекламы 1-5. <br>
(Здесь и далее отнесение заказов к источнику рекламы производится на основании источника первого перехода пользователя, совершившего заказ).

In [ ]:
print(f'Сумма затрат за период {costs["costs"].sum()}')
print()
print('Распределение затрат по источникам')
display(costs.groupby('source_id')['costs'].sum())

<div style="border:solid grey 2px; padding: 30px">
Маркетинговые затраты составили 329 млн. руб. В основном они были распределены аналогично выручке между источникми рекламы 1-5, но наибольшие затраты (141 млн руб.) были на источник 3, наименьшие (из этих 5 источников) на источник 1 (20 млн руб.).
